### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
23,39462903,122720,79085,1345,7,60,1006,1,2022-10-16 21:58,2022-10-16 21:58,0,1,11793,3,1
24,39462915,79085,10246,1345,8,60,1006,4,2022-10-19 21:58,2022-10-19 21:58,0,1,5500,3,1
25,39627969,10653,79085,3,0,0,1006,6,2022-10-21 21:59,2022-10-21 21:59,0,0,6179,1,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

263

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,122720,33270,34432,0,6,1,0,0,F 22 01,8,7,6,39462903
1,79085,36686,35524,0,6,2,1,0,5-4-1a,7,6,7,39462903
0,79085,47023,32630,3,0,3,0,0,4-2-3-1a,6,7,8,39462915
1,10246,22943,37336,0,2,1,0,0,5311Separ2020,7,9,10,39462915
0,10653,41443,35822,1,2,5,2,0,ala,6,7,4,39627969
1,79085,29779,35400,1,8,3,2,0,5-3-2b,5,7,7,39627969


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

526

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2076

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,88984,8,22,7,1,0,65,1,22564065007012,2076
1,142717,8,22,7,1,0,53,2,22551053007005,2076
2,122540,8,18,5,3,0,44,2,18542044005010,2076
3,120765,8,18,6,0,2,30,16,18514030006007,2076
4,78473,8,15,5,0,3,31,12,15519031005008,2076
5,142636,8,15,5,0,3,17,24,15493017005004,2076
6,79853,8,10,3,1,4,18,19,10499018003006,2076
7,142663,8,9,3,0,5,12,30,9482012003003,2076
8,47955,8,6,2,0,6,7,36,6471007002001,2076
9,123911,8,3,1,0,7,6,48,3458006001009,2076


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

2

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()